In [50]:
from qiskit import *
# from qiskit.providers.ibmq import least_busy
# from qiskit.tools.monitor import job_monitor

In [ ]:
class QBitGen:
    def __init__(self):
        self.__random_gen = ""
        self.__qrn_circ = None
        self.__failed_results = False
        self.__provider = None

    # Setup the circuit 
    def init_circ(self):
        qcirc = QuantumCircuit(3, 3)

        # Create a state of 3 entangled qbits
        qcirc.h(0)
        qcirc.cx(0, 2)
        qcirc.barrier()

        qcirc.h(1)
        qcirc.cx(1, 2)
        qcirc.barrier()

        # Perform measurements in z-basis
        qcirc.measure(0, 0)
        qcirc.measure(1, 1)
        qcirc.measure(2, 2)

        self.__qrn_circ = qcirc
        
    # Run the circuit in simulator
    def simulate_circ(self, num_shots = 1024):
        if(self.__provider == None):
            print("Load the IBMQ account first please")
        
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(qrn_circ, shots = num_shots, memory=True)
        result = aer_sim.run(qobj).result().get_memory()

        # Verify XOR-rule for obtained results
        
        random_gen = ""
        for measurement in result:
            if int(measurement[0]) ^ int(measurement[1]) != int(measurement[2]):
                print("Bad result found: " + measurement)
                self.__failed_results = True
                return

            # Append the measurement
            random_gen += measurement[0]

        self.__failed_results = False
        self.__random_gen = random_gen
    
    # Load the IBMQ account, assumes account token has been alreadu saved for access to hardware
    def load_provider(self):
        try:
            IBMQ.load_account()
            self.__provider = IBMQ.get_provider(hub='ibm-q')
        except IBMQProviderError:
            print("Could not fetch providers")
    
    def get_random(self)->str:
        